In [2]:
import glob 
import time
import math
import random
import numpy as np
import pandas as pd
from scipy import linalg as LA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import NuSVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import metrics
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from termcolor import colored
from matplotlib import colors
import pickle
import os

## Reading cleaned csv files

### User i is data_1$[i]$ in the following

In [3]:
data_1 = [0] * 57
for i in range(len(data_1)):
    fnames = glob.glob('labeled csv Geolife/'+str(i)+'/*.csv')
    data_1[i] = np.array([np.loadtxt(f, delimiter=',')[1:] for f in fnames])
data_1 = np.array(data_1)

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """


In [4]:
fnames = glob.glob('labeled csv Geolife/**/*.csv')
len(fnames)

3572

### Users are stacked together in data_2 below

In [5]:
data_2 = []
fnames = glob.glob('labeled csv Geolife/**/*.csv')
for f in fnames:
    data_2.append(np.loadtxt(f, delimiter=',')[1:])
data_2 = np.array(data_2)

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """


### Removing segments with length less than 1e-10 because of numerical precision

In [6]:
data_3 = [0] * len(data_2)
h = 1e-12
c = 0
for i in range(len(data_2)):
    p1 = data_2[i][:-1]
    p2 = data_2[i][1:]
    L = ((p2[:,:2]-p1[:,:2])*(p2[:,:2]-p1[:,:2])).sum(axis =1)
    I = np.where(L > h)[0]
    J = np.where(L < h)[0]
    if len(J) > 0:
        c += 1
    p1 = p1[I]
    p2 = p2[I]
    if len(I) == 0:
        print(i)
    gamma = np.concatenate((p1, p2[-1].reshape(1,4)), 0) 
    if len(gamma) > 0:
        data_3[i] = gamma
    data_3[i] = np.array(data_3[i])
data_3 = np.array(data_3)
c

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


2811

# Partitioning trajectories to less than 20 minutes long

In [7]:
def partition(trajectory, threshold=20):
    trajectories = []
    # ("24 * 60 *" makes days_date to minutes)
    Time = 24 * 60 * (trajectory[:,2][1:] - trajectory[:,2][:-1]) # : x minutes 
    J = np.where(Time > threshold)[0]
    J = J.tolist()
    if len(J) == 0:
        trajectories.append(trajectory)
    else:
        J = [0] + J + [len(trajectory)]
        for j in range(len(J) - 1):
            trajectories.append(trajectory[J[j]:J[j+1]])
    return trajectories

### data_4 below is the array of trajectories having less than 20 minutes long

In [8]:
data_4 = []
for i in range(len(data_3)):
    A = partition(data_3[i], threshold=20)
    for j in range(len(A)):
        data_4.append(A[j])
data_4 = np.array(data_4, dtype='object')
len(data_4)

5302

In [9]:
I = np.where(np.array([len(data_4[i]) for i in range(len(data_4))]) >= 10)[0]
data_4 = data_4[I]
len(data_4)

5174

In [10]:
int1 = np.vectorize(int)
data_5 = []
c = 0
for i in range(len(data_4)):
    if len(set(int1(data_4[i][:,3]))) < 2: 
        data_5.append(data_4[i])
        c += 1
data_5 = np.array(data_5)
c

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


3567

In [11]:
Modes = ['walk', 'bike', 'bus', 'driving', 'train']

# Separating transportation modes for each trip

In [12]:
data = []
c = np.zeros(5)

Len = np.array([len(set(int1(data_4[i][:,3]))) for i in range(len(data_4))])

K = np.where(Len == 1)[0]
c[0] = len(K)
for k in K:
    data.append(data_4[k])

for k in range(2,3):
    K = np.where(Len == k)[0]
    c[k] = len(K)
    data_temp = data_4[K]
    for i in range(len(data_temp)):
        S = set(data_temp[i][:,3])
        for s in S:
            I = np.where(data_temp[i][:,3] == s)[0]
            if (max(I) - min(I) + 1 == len(I)):
                data.append(data_temp[i][I])
            else:
                J = np.where(np.array([I[j+1]-I[j] for j in range(len(I)-1)]) != 1)[0]
                if J[0] >= 10:
                    data.append(data_temp[i][J[:J[0]]])
                for j in range(len(J)-1):
                    if (J[j+1]-J[j]) >= 10:
                        data.append(data_temp[i][J[J[j]:J[j+1]]])

data = np.array(data)
print(len(data))

6404


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [13]:
Lengths = np.array([len(data[i]) for i in range(len(data))])
idx = np.where(Lengths >= 10)[0]
data = data[idx]

In [14]:
Labels_sizes = np.array([list(set(data[i][:,3]))[0] for i in range(len(data))])
for k in range(5):
    print("The number of trajectries with label", k, 'is:', len(np.where(Labels_sizes==k)[0]))

The number of trajectries with label 0 is: 2060
The number of trajectries with label 1 is: 1110
The number of trajectries with label 2 is: 1094
The number of trajectries with label 3 is: 975
The number of trajectries with label 4 is: 359


# Multi class classification with different featuerizations

In [15]:
import trjtrypy as tt
from trjtrypy.distances import d_Q
from trjtrypy.distances import d_Q_pi
import trjtrypy.visualizations as vs
from scipy.spatial import distance
from trjtrypy.featureMappings import curve2vec


def ExpCurve2Vec(points,curves,mu):
    D = tt.distsbase.DistsBase()
    a = np.array([np.exp(-1*np.power(D.APntSetDistACrv(points,curve),2)/(mu)**2) for curve in curves])
    return a

## Features

In [16]:
# length calculator (for trajectories of one transportation mode can be used)
def length(x):
    p1 = x[:,:2][:-1]
    p2 = x[:,:2][1:]
    L = np.sqrt(((p2-p1)*(p2-p1)).sum(axis =1))
    Length = [sum(L)/len(L)]
    return Length

In [17]:
# speed calculator (for trajectories of one transportation mode can be used)
def speed(x):
    t = x[:,2][1:] - x[:,2][:-1] + 1e-10
    p1 = x[:,:2][:-1]
    p2 = x[:,:2][1:]
    L = np.sqrt(((p2-p1)*(p2-p1)).sum(axis =1))
    speeds = L/t
    s = [sum(speeds)/len(speeds)]
    return s

In [18]:
# acceleration calculator (for trajectories of one transportation mode can be used)
def acceleration(x):
    t = x[:,2][1:] - x[:,2][:-1] + 1e-10
    p1 = x[:,:2][:-1]
    p2 = x[:,:2][1:]
    L = np.sqrt(((p2-p1)*(p2-p1)).sum(axis =1))
    accelerations = L/t**2
    a = [1e-5 * sum(accelerations)/len(accelerations)]
    return a

In [19]:
# jerk calculator (for trajectories of one transportation mode can be used)
def jerk(x):
    t = x[:,2][1:] - x[:,2][:-1] + 1e-10
    p1 = x[:,:2][:-1]
    p2 = x[:,:2][1:]
    L = np.sqrt(((p2-p1)*(p2-p1)).sum(axis =1))
    jerks = L/t**3
    a = [1e-5 * sum(jerks)/len(jerks)]
    return a

# Classifiers

In [97]:
clf1 = [make_pipeline(StandardScaler(), LinearSVC(dual=False, C=1e10, tol=1e-5, 
                               class_weight='balanced', max_iter=1000)), 
        "Linear SVM, C="+str(1e10)]
clf2 = [make_pipeline(StandardScaler(), svm.SVC(C=1000, kernel='rbf', gamma=1000, max_iter=200000)),
        "Gaussian SVM, C="+str(1000)+", gamma ="+str(1000)]
clf3 = [make_pipeline(StandardScaler(), svm.SVC(C=1000, kernel='poly', degree=3, max_iter=400000)),
        "Poly kernel SVM, C="+str(1000)+", deg=3"]
clf4 = [DecisionTreeClassifier(), "Decision Tree"]
clf5 = [RandomForestClassifier(n_estimators=100), "RandomForestClassifier, n="+str(100)]
clf6 = [KNeighborsClassifier(n_neighbors=5), "KNN"]
clf7 = [LogisticRegression(solver='newton-cg'), "Logistic Regression"]

clf = [clf1, clf2, clf3, clf4, clf5, clf6, clf7]
classifs = [item[0] for item in clf]
keys = [item[1] for item in clf]

In [74]:
data_traj = np.array([data[i][:,:2] for i in range(len(data))], dtype='object')

# Classification

## Classification with only Physical features

In [75]:
def select_physical_features(data, Leng=True, spd=True, accn=True, jrk=True):
    Start_time = time.time()
    
    n = len(data)
    labels = np.array([data[i][:,3][0] for i in range(n)]).reshape(-1,1)
    data_traj = np.array([data[i][:,:2] for i in range(n)])
    length_ = np.array([length(data_traj[i]) for i in range(n)]).reshape(-1,1)
    speed_ = np.array([speed(data[i]) for i in range(n)]).reshape(-1,1)
    acceleration_ = np.array([acceleration(data[i]) for i in range(n)]).reshape(-1,1)
    jerk_ = np.array([jerk(data[i]) for i in range(n)]).reshape(-1,1)
    
    A = np.concatenate((length_, speed_, acceleration_, jerk_, labels), 1)

    I = []
    if Leng == True:
        I.append(0)
    if spd == True:
        I.append(1)
    if accn == True:
        I.append(2)
    if jrk == True:
        I.append(3)
    I.append(-1)
    
    print(colored(f"Total time for mapping row data: {time.time() - Start_time}", 'green'))
    return A[:,I]

In [76]:
def classification_physical(data, Leng=True, spd=True, accn=True, jrk=True, 
                            num_trials=10, classifiers=clf):

    Start_time = time.time()
    models = [item[0] for item in classifiers]
    keys = [item[1] for item in classifiers]
    r = len(classifiers)
    train_error_mean = np.zeros(r)
    test_error_mean = np.zeros(r)
    test_error_std = np.zeros(r)
    train_error_list = np.zeros((r, num_trials,)) 
    test_error_list = np.zeros((r, num_trials))

    for s in range(num_trials):
        
        Data = select_physical_features(data, Leng=Leng, spd=spd, accn=accn, jrk=jerk)
        X_train, X_test, y_train, y_test = train_test_split(Data[:,:-1], Data[:,-1], 
                                                            test_size=0.2)
        I = np.arange(len(X_train))
        np.random.shuffle(I)
        train_data = X_train[I]
        train_labels = y_train[I]

        J = np.arange(len(X_test))
        np.random.shuffle(J)
        test_data = X_test[J]
        test_labels = y_test[J]
        
        for k in range(r):            
            model = models[k]

            #Train the model using the training sets
            model.fit(train_data, train_labels)

            #Predict train labels
            train_pred = model.predict(train_data)
            err = 1 - metrics.accuracy_score(train_labels, train_pred)
            train_error_list[k][s] = err
            
            #Predict test labels
            test_pred = model.predict(test_data)
            er = 1 - metrics.accuracy_score(test_labels, test_pred)
            test_error_list[k][s] = er
            
    for k in range(r):
        train_error_mean[k] = np.mean(train_error_list[k])
        test_error_mean[k] = np.mean(test_error_list[k])
        test_error_std[k] = np.std(test_error_list[k])
    
    Dic = {}
    for k in range(len(keys)): 
        Dic[k] = [keys[k], np.round(train_error_mean[k], decimals = 4), 
                    np.round(test_error_mean[k], decimals = 4),
                    np.round(test_error_std[k], decimals = 4)]

    pdf = pd.DataFrame.from_dict(Dic, orient='index', columns=['Classifier','Train Error', 
                                                               'Test Error', 'Std Error'])
    print(colored(f"Total time: {time.time() - Start_time}", 'red'))
    return pdf

### Classification with just physical features

In [77]:
classification_physical(data, Leng=True, spd=True, accn=True, jrk=True, num_trials=50, 
                        classifiers=clf)

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


Total time for mapping row data: 2.171159029006958


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=200000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/sit

Total time for mapping row data: 2.3183751106262207


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=200000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/sit

...

Total time for mapping row data: 1.9914019107818604


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=200000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Total time: 654.409509897232


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:437: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


,Classifier,Train Error,Test Error,Std Error
0,"Linear SVM, C=10000000000.0",0.4035,0.3978,0.0137
1,"Gaussian SVM, C=1000, gamma =1000",0.2635,0.2906,0.0134
2,"Poly kernel SVM, C=1000, deg=3",0.7347,0.7349,0.0927
3,Decision Tree,0.0000,0.2915,0.0112
4,"RandomForestClassifier, n=100",0.0001,0.2273,0.0097
5,KNN,0.1942,0.2464,0.0106
6,Logistic Regression,0.7452,0.7449,0.1371


## Classification with generating random Q in each iteration

In [78]:
def select_features_Q(data, version='unsigned', sigma=1, Q_size=20, Leng=False, 
                      spd=False, accn=False, jrk=False, normal=False):
    Start_time = time.time()
    
    n = len(data)
    labels = np.array([data[i][:,3][0] for i in range(n)]).reshape(-1,1)
    data_traj = np.array([data[i][:,:2] for i in range(n)])

    Min = np.min([np.min(data_traj[i], 0) for i in range(n)], 0)
    Max = np.max([np.max(data_traj[i], 0) for i in range(n)], 0)
    Mean = np.mean([np.mean(data_traj[i], 0) for i in range(n)], 0)
    Std = np.std([np.std(data_traj[i], 0) for i in range(n)], 0)
    
    Q = np.ones((Q_size,2))
    Q[:,0] = 0.5 * np.random.random_sample(Q_size) + Mean[0] + 0.1
    Q[:,1] = 0.5 * np.random.random_sample(Q_size) + Mean[1] + 0.6

    projected = np.array(curve2vec(Q, data_traj, version=version, sigma=sigma))
    length_ = np.array([length(data_traj[i]) for i in range(n)]).reshape(-1,1)
    speed_ = np.array([speed(data[i]) for i in range(n)]).reshape(-1,1)
    acceleration_ = np.array([acceleration(data[i]) for i in range(n)]).reshape(-1,1)
    jerk_ = np.array([jerk(data[i]) for i in range(n)]).reshape(-1,1)
    
    A = np.concatenate((projected, length_, speed_, acceleration_, jerk_, labels), 1)

    I = list(np.arange(Q_size))
    if Leng == True:
        I.append(Q_size)
    if spd == True:
        I.append(Q_size+1)
    if accn == True:
        I.append(Q_size+2)
    if jrk == True:
        I.append(Q_size+3)
    I.append(-1)
    
    if normal == True:
        A[:,I][:-1] = (A[:,I][:-1]-np.mean(A[:,I][:-1], 0))/(np.std(A[:,I][:-1], 0))
    
    print(colored(f"Total time for mapping row data: {time.time() - Start_time}", 'green'))
    return A[:,I]

In [79]:
def classification_Q(data, version='unsigned', sigma=1, Q_size=20, Leng=True, spd=True,
                     accn=True, jrk=True, num_trials=10, classifiers=clf, normal=False):

    Start_time = time.time()
    models = [item[0] for item in classifiers]
    keys = [item[1] for item in classifiers]
    r = len(classifiers)
    train_error_mean = np.zeros(r)
    test_error_mean = np.zeros(r)
    test_error_std = np.zeros(r)
    train_error_list = np.zeros((r, num_trials,)) 
    test_error_list = np.zeros((r, num_trials))

    for s in range(num_trials):
        
        Data = select_features_Q(data, version, sigma=sigma, Q_size=Q_size, Leng=Leng, 
                                 spd=spd, accn=accn, jrk=jerk)

        X_train, X_test, y_train, y_test = train_test_split(Data[:,:-1], Data[:,-1], 
                                                            test_size=0.2)
        
        I = np.arange(len(X_train))
        np.random.shuffle(I)
        train_data = X_train[I]
        train_labels = y_train[I]

        J = np.arange(len(X_test))
        np.random.shuffle(J)
        test_data = X_test[J]
        test_labels = y_test[J]
        
        for k in range(r):            
            model = models[k]

            #Train the model using the training sets
            model.fit(train_data, train_labels)

            #Predict train labels
            train_pred = model.predict(train_data)
            err = 1 - metrics.accuracy_score(train_labels, train_pred)
            train_error_list[k][s] = err
            
            #Predict test labels
            test_pred = model.predict(test_data)
            er = 1 - metrics.accuracy_score(test_labels, test_pred)
            test_error_list[k][s] = er
            
    for k in range(r):
        train_error_mean[k] = np.mean(train_error_list[k])
        test_error_mean[k] = np.mean(test_error_list[k])
        test_error_std[k] = np.std(test_error_list[k])
    
    Dic = {}

    for k in range(len(keys)): 
        Dic[k] = [keys[k], np.round(train_error_mean[k], decimals = 4), 
                    np.round(test_error_mean[k], decimals = 4),
                    np.round(test_error_std[k], decimals = 4)]

    pdf = pd.DataFrame.from_dict(Dic, orient='index', columns=['Classifier','Train Error', 
                                                               'Test Error', 'Std Error'])
    print(colored(f"Total time: {time.time() - Start_time}", 'red'))
    return pdf

### Classification with just $v_Q$ with random Q in each iteration

In [80]:
classification_Q(data, version='unsigned', Q_size=20, Leng=False, spd=False,
                 accn=False, jrk=False, num_trials=50, classifiers=clf, normal=False)

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


Total time for mapping row data: 10.66481900215149


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


Total time for mapping row data: 10.584946155548096


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


...

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


Total time for mapping row data: 10.40149974822998


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Total time: 1026.3558979034424


,Classifier,Train Error,Test Error,Std Error
0,"Linear SVM, C=10000000000.0",0.4893,0.4959,0.0289
1,"Gaussian SVM, C=1000, gamma =1000",0.1736,0.2313,0.0126
2,"Poly kernel SVM, C=1000, deg=3",0.6286,0.6332,0.0141
3,Decision Tree,0.0000,0.2418,0.0143
4,"RandomForestClassifier, n=100",0.0000,0.1808,0.0120
5,KNN,0.1625,0.2241,0.0131
6,Logistic Regression,0.6116,0.6183,0.0233


### Classification with just $v_Q^{\varsigma}$ with random Q in each iteration

In [81]:
classification_Q(data, version='signed', sigma=1, Q_size=20, Leng=False, spd=False,
                 accn=False, jrk=False, num_trials=50, classifiers=clf, normal=False)

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


Total time for mapping row data: 13.179633140563965


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


Total time for mapping row data: 13.080919027328491


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


...

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


Total time for mapping row data: 13.04663610458374


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Total time: 2051.2437648773193


,Classifier,Train Error,Test Error,Std Error
0,"Linear SVM, C=10000000000.0",0.6592,0.6772,0.0141
1,"Gaussian SVM, C=1000, gamma =1000",0.0294,0.5711,0.0150
2,"Poly kernel SVM, C=1000, deg=3",0.4285,0.6537,0.0225
3,Decision Tree,0.0264,0.4167,0.0126
4,"RandomForestClassifier, n=100",0.0265,0.3022,0.0120
5,KNN,0.3673,0.5271,0.0148
6,Logistic Regression,0.6247,0.6282,0.0150


### Classification with $v_Q^+$ with random Q in each iteration

In [88]:
classification_Q(data, version='unsigned', Q_size=20, Leng=True, spd=True,
                 accn=True, jrk=True, num_trials=50, classifiers=clf, normal=False)

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


Total time for mapping row data: 10.616398811340332


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:437: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line 

Total time for mapping row data: 10.534627199172974


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:437: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line 

...

Total time for mapping row data: 13.089530229568481


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Total time: 1244.3101139068604


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:437: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


,Classifier,Train Error,Test Error,Std Error
0,"Linear SVM, C=10000000000.0",0.2798,0.2869,0.0139
1,"Gaussian SVM, C=1000, gamma =1000",0.0762,0.2068,0.0126
2,"Poly kernel SVM, C=1000, deg=3",0.6264,0.6235,0.0326
3,Decision Tree,0.0000,0.1973,0.0116
4,"RandomForestClassifier, n=100",0.0000,0.1189,0.0094
5,KNN,0.1620,0.2086,0.0105
6,Logistic Regression,0.6558,0.6564,0.1775


### Classification with $v_Q^{\varsigma +}$ with random Q in each iteration

In [83]:
classification_Q(data, version='signed', sigma=1, Q_size=20, Leng=True, spd=True,
                 accn=True, jrk=True, num_trials=50, classifiers=clf, normal=False)

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


Total time for mapping row data: 13.20738697052002


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nest

Total time for mapping row data: 13.16734504699707


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:437: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line 

...

Total time for mapping row data: 12.69602108001709


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Total time: 1778.2128159999847


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:437: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


,Classifier,Train Error,Test Error,Std Error
0,"Linear SVM, C=10000000000.0",0.3971,0.4059,0.0138
1,"Gaussian SVM, C=1000, gamma =1000",0.0117,0.5696,0.0145
2,"Poly kernel SVM, C=1000, deg=3",0.2607,0.5379,0.0149
3,Decision Tree,0.0000,0.2688,0.0123
4,"RandomForestClassifier, n=100",0.0000,0.1642,0.0100
5,KNN,0.1943,0.2611,0.0111
6,Logistic Regression,0.6837,0.6840,0.1552


## Classification with $v_Q^+$ with choosing Q in each iteration by the perceptron-like algorithm

This didn't work better than random choice of Q

In [89]:
def get_mu_(data_1, data_2):
    a = np.mean([np.mean(data_1[i], 0) for i in range(len(data_1))], 0)
    b = np.mean([np.mean(data_2[i], 0) for i in range(len(data_2))], 0)
    c = abs(a-b)
    return max(c)


modes = [0, 1, 2, 3, 4]
def get_mu(train):
    mus = []
    train_traj = np.array([train[i][:,:2] for i in range(len(train))])
    for i in range(len(modes)-1):
        for j in range(i+1, len(modes)):
            I = np.where(np.array([train[k][0][-1] for k in range(len(train))]) == i)[0]
            J = np.where(np.array([train[k][0][-1] for k in range(len(train))]) == j)[0]
            mus.append(get_mu_(train_traj[I], train_traj[J]))
    return np.max(mus)

In [90]:
from scipy.stats import entropy

def initialize_Q(train, C, gamma, Q_size, mu_coeff, model, Leng=True, spd=True, accn=True, 
                 jrk=True, n_estimators=100, n_neighbors=5, deg=11):
    Start_time = time.time()
    n = len(train)
    train_traj = np.array([train[i][:,:2] for i in range(n)])
    train_labels = np.array([list(set(train[i][:,-1]))[0] for i in range(n)])
    length_ = np.array([length(train_traj[i]) for i in range(n)]).reshape(-1,1)
    speed_ = np.array([speed(train[i]) for i in range(n)]).reshape(-1,1)
    acceleration_ = np.array([acceleration(train[i]) for i in range(n)]).reshape(-1,1)
    jerk_ = np.array([jerk(train[i]) for i in range(n)]).reshape(-1,1)
    
# Make the classifier
    if model == 'LSVM':
        Model = make_pipeline(LinearSVC(dual=False, C=C, tol=1e-5, class_weight='balanced', max_iter=1000))
    elif model == 'GSVM':
        Model = make_pipeline(StandardScaler(), svm.SVC(C=C, kernel='rbf', gamma=gamma, max_iter=200000))
    elif model == 'PSVM':
        Model = make_pipeline(StandardScaler(), svm.SVC(C=C, kernel='poly', degree=deg, max_iter=400000))
    elif model == 'DT':
        Model = DecisionTreeClassifier()
    elif model == 'RF':
        Model = RandomForestClassifier(n_estimators=n_estimators)
    elif model == 'KNN':
        Model = KNeighborsClassifier(n_neighbors=n_neighbors)
    elif model == 'LR':
        Model = LogisticRegression(solver='lbfgs')
    else:
        print("model is not supported")
        
    #print(train.shape, train[0].shape)
    mu = get_mu(train) * mu_coeff
    std = mu/2
    errors = []
    Q = []
    Train_0 = np.concatenate((length_, speed_, acceleration_, jerk_), 1)
    #Train_0 = length_
    Model.fit(Train_0, train_labels)
    train_pred = Model.predict(Train_0) 
    error = 1 - metrics.accuracy_score(train_labels, train_pred)
    errors.append(error)
    
    #I = np.where(train_pred != train_labels)[0]
    I = np.arange(len(train_labels))
    if model in ['LSVM', 'GSVM', 'LR', 'PSVM']:
        J = np.max(Model.decision_function(Train_0[I]), 1)
    elif model in ['DT', 'RF', 'KNN']:
        J = np.max(entropy(Model.predict_proba(Train_0[I]), axis=1))

    index = I[np.argmax(J)]
    k = np.random.randint(0, high=len(train_traj[index]))
    q = train_traj[index][k] + np.random.normal(0, std, 2)
    Q.append(q)

# Iteratively choose landmarks
    for i in range(1, Q_size):
        Train = np.concatenate((Train_0, np.array(ExpCurve2Vec(np.array(Q), train_traj, mu))), 1)
        Model.fit(Train, train_labels)
        train_pred = Model.predict(Train)
        
        error = 1 - metrics.accuracy_score(train_labels, train_pred)
        errors.append(error)
        
        #K = np.where(train_pred != train_labels)[0] # was I previously
        I = np.arange(len(train_labels))
        #if len(K) == 0: 
        #    return np.array(Q), mu, np.array(errors), error
        if model in ['LSVM', 'GSVM', 'LR', 'PSVM']:
            J = np.max(Model.decision_function(Train[I]), 1)
        elif model in ['DT', 'RF', 'KNN']:
            J = np.max(entropy(Model.predict_proba(Train[I]), axis=1))
        
        index = I[np.argmax(J)]
        k = np.random.randint(0, high=len(train_traj[index]))
        q = train_traj[index][k] + np.random.normal(0, std, 2)
        Q.append(q)

    Train = np.concatenate((Train_0, np.array(ExpCurve2Vec(np.array(Q), train_traj, mu))), 1)
    Model.fit(Train, train_labels)
    train_pred = Model.predict(Train)

    error = 1 - metrics.accuracy_score(train_labels, train_pred)
    errors.append(error)

    print(colored(f"Total time for mapping row data: {time.time() - Start_time}", 'green'))

    return np.array(Q), mu, np.array(errors), error

In [91]:
def classification_init_Q(data, C, gamma, Q_size, mu_coeff, model, epoch, init_iter, 
                          classifiers, Leng=True, spd=True, accn=True, jrk=True, 
                          n_estimators=100, n_neighbors=5, deg=11):

    start_time = time.time()
    models = [item[0] for item in classifiers]
    keys = [item[1] for item in classifiers]

    r = len(classifiers)

    train_error_mean = np.zeros(r)
    test_error_mean = np.zeros(r)
    test_error_std = np.zeros(r)
    
    train_errors = np.zeros((r, epoch)) 
    test_errors = np.zeros((r, epoch))

    n = len(data) 
    data_traj = np.array([data[i][:,:2] for i in range(n)])
    labels = np.array([list(set(data[i][:,-1]))[0] for i in range(n)])
    length_ = np.array([length(data_traj[i]) for i in range(n)]).reshape(-1,1)

    for s in range(epoch):
        I = np.arange(n)
        np.random.shuffle(I)
        train = data_traj[I][:int(0.8 * n)]
        test = data_traj[I][int(0.8 * n):]
        train_labels = labels[I][:int(0.8 * n)]
        test_labels = labels[I][int(0.8 * n):]
        
        x_preds = np.zeros((r, len(train)))
        y_preds = np.zeros((r, len(test)))

        Q_list = []
        temp_errors = []
        mu_list = []
        for j in range(init_iter):
            B = initialize_Q(data[I][:int(0.8 * n)], C=C, gamma=gamma, Q_size=Q_size, 
                             mu_coeff=mu_coeff, model=model, Leng=Leng, spd=spd, 
                             accn=accn, jrk=jrk, n_estimators=n_estimators, 
                             n_neighbors=n_neighbors, deg=deg)
            
            Q_list.append(B[0])
            temp_errors.append(B[-1])
            mu_list.append(B[1])

        h = np.argmin(temp_errors)
        Q = Q_list[h]
        mu = mu_list[h]
        print("mu =", mu)

        train_data = np.concatenate((length_[I][:int(0.8 * n)], 
                                     np.array(ExpCurve2Vec(Q, train, mu))), 1)
        test_data = np.concatenate((length_[I][int(0.8 * n):], 
                                    np.array(ExpCurve2Vec(Q, test, mu))), 1)

        for k in range(r): 
            Model = models[k]
            Model.fit(train_data, train_labels)
            x_preds[k] = Model.predict(train_data)                
            y_preds[k] = Model.predict(test_data)

        for k in range(r):
            train_errors[k][s] = 1 - metrics.accuracy_score(train_labels, x_preds[k])
            test_errors[k][s] = 1 - metrics.accuracy_score(test_labels, y_preds[k])
            
    for k in range(r):
        train_error_mean[k] = np.mean(train_errors[k])
        test_error_mean[k] = np.mean(test_errors[k])
        test_error_std[k] = np.std(test_errors[k])
    
    Dict = {}

    for k in range(len(keys)): 
        Dict[k+1] = [keys[k], np.round(train_error_mean[k], decimals=4), 
                     np.round(test_error_mean[k], decimals=4),
                     np.round(test_error_std[k], decimals=4)]

    pdf = pd.DataFrame.from_dict(Dict, orient='index', 
                columns=['Classifier','Train Error', 'Test Error', 'Std Error'])
    
    print(colored(f"total time = {time.time() - start_time}", "red"))

    return pdf, train_error_mean, test_error_mean

In [93]:
L = classification_init_Q(data, C=1e10, gamma=1, Q_size=20, mu_coeff=1, model='LSVM', 
                          epoch=50, init_iter=3, classifiers=clf, Leng=True, spd=True, 
                          accn=True, jrk=True, n_estimators=100, n_neighbors=5)
L[0]

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must

Total time for mapping row data: 97.7084391117096


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


Total time for mapping row data: 95.25634694099426


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


...

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


Total time for mapping row data: 97.10856485366821
mu = 4.2276461786581905


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


total time = 3207.7808039188385


,Classifier,Train Error,Test Error,Std Error
1,"Linear SVM, C=10000000000.0",0.4330,0.4416,0.0178
2,"Gaussian SVM, C=1000, gamma =1000",0.0219,0.2527,0.0212
3,"Poly kernel SVM, C=1000, deg=3",0.5932,0.6058,0.0173
4,Decision Tree,0.0000,0.2570,0.0223
5,"RandomForestClassifier, n=100",0.0001,0.1882,0.0256
6,KNN,0.1776,0.2496,0.0227
7,Logistic Regression,0.6301,0.6315,0.0113


In [54]:
G = classification_init_Q(data, C=1e3, gamma=1e2, Q_size=20, mu_coeff=1, model='GSVM', 
                          epoch=50, init_iter=3, classifiers=clf, Leng=True, spd=True, 
                          accn=True, jrk=True, n_estimators=100, n_neighbors=5)
G[0]

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specif

Total time for mapping row data: 168.57351183891296


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=200000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, Convergenc

Total time for mapping row data: 169.52408170700073


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=200000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, Convergenc

...

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=200000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, Convergenc

Total time for mapping row data: 174.48254799842834
mu = 3.5705820363379956


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=200000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


total time = 5200.662418842316


,Classifier,Train Error,Test Error,Std Error
1,KNN,0.1578,0.2179,0.0102
2,"Linear SVM, C=1000.0",0.4245,0.4269,0.0337
3,"Linear SVM (Standardized), C = 1000.0",0.3946,0.3964,0.0317
4,"Gaussian SVM, C=10000.0, gamma =1",0.1560,0.2244,0.0083
5,"Gaussian SVM, C=10, gamma =auto",0.4179,0.4254,0.0477
6,"Gaussian SVM, C=1000.0, gamma =1000.0",0.0089,0.2251,0.0160
7,"Gaussian SVM, C=10000.0, gamma =10000.0",0.0008,0.3325,0.0167
8,"Gaussian SVM, C=10000.0, gamma =30000.0",0.0003,0.3818,0.0195
9,Logistic Regression,0.6312,0.6315,0.0129
10,Decision Tree,0.0000,0.2342,0.0134


In [92]:
P = classification_init_Q(data, C=1e3, gamma=1, Q_size=20, mu_coeff=1, model='PSVM', 
                          epoch=50, init_iter=3, classifiers=clf, Leng=True, spd=True, 
                          accn=True, jrk=True, n_estimators=100, n_neighbors=5, deg=11)
P[0]

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must

Total time for mapping row data: 486.92257165908813


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, Con

Total time for mapping row data: 483.4290359020233


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, Con

...

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, Con

Total time for mapping row data: 446.30090498924255
mu = 4.671042504773254


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=200000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


total time = 14670.345854997635


,Classifier,Train Error,Test Error,Std Error
1,"Linear SVM, C=10000000000.0",0.7338,0.7312,0.0538
2,"Gaussian SVM, C=1000, gamma =1000",0.4452,0.4718,0.0384
3,"Poly kernel SVM, C=1000, deg=3",0.6297,0.6292,0.0114
4,Decision Tree,0.2002,0.4931,0.0571
5,"RandomForestClassifier, n=100",0.1880,0.4815,0.0771
6,KNN,0.3484,0.4848,0.0704
7,Logistic Regression,0.6315,0.6299,0.0111


In [106]:
D = classification_init_Q(data, C=1, gamma=1, Q_size=20, mu_coeff=1, model='DT', 
                          epoch=50, init_iter=3, classifiers=clf, Leng=True, spd=True, 
                          accn=True, jrk=True, n_estimators=100, n_neighbors=5)
D[0]

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must

Total time for mapping row data: 113.36778903007507


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


Total time for mapping row data: 113.62630796432495


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


...

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


Total time for mapping row data: 104.25990724563599
mu = 3.9298992955956464


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


total time = 3296.4183111190796


,Classifier,Train Error,Test Error,Std Error
1,"Linear SVM, C=1000.0",0.4213,0.4229,0.0260
2,"Gaussian SVM, C=1000.0, gamma =1000.0",0.0109,0.2437,0.0205
3,Decision Tree,0.0000,0.2289,0.0131
4,"RandomForestClassifier, n=100",0.0008,0.1525,0.0098
5,KNN,0.1597,0.2187,0.0112
6,Logistic Regression,0.6312,0.6242,0.0106
7,"AdaBoost, n=100",0.3272,0.3452,0.0263


In [107]:
R = classification_init_Q(data, C=1, gamma=1, Q_size=20, mu_coeff=1, model='RF', 
                          epoch=50, init_iter=3, classifiers=clf, Leng=True, spd=True, 
                          accn=True, jrk=True, n_estimators=100, n_neighbors=5)
R[0]

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must

Total time for mapping row data: 138.96438002586365


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


Total time for mapping row data: 126.35521483421326


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


...

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


Total time for mapping row data: 122.52555298805237
mu = 3.868722599601412


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


total time = 3839.230007171631


,Classifier,Train Error,Test Error,Std Error
1,"Linear SVM, C=1000.0",0.4188,0.4283,0.0243
2,"Gaussian SVM, C=1000.0, gamma =1000.0",0.0119,0.2370,0.0084
3,Decision Tree,0.0000,0.2318,0.0081
4,"RandomForestClassifier, n=100",0.0002,0.1546,0.0085
5,KNN,0.1580,0.2202,0.0117
6,Logistic Regression,0.6308,0.6338,0.0093
7,"AdaBoost, n=100",0.3369,0.3458,0.0264


In [108]:
K = classification_init_Q(data, C=1, gamma=1, Q_size=20, mu_coeff=1, model='KNN', 
                          epoch=50, init_iter=3, classifiers=clf, Leng=True, spd=True, 
                          accn=True, jrk=True, n_estimators=100, n_neighbors=5)
K[0]

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must

Total time for mapping row data: 114.73997902870178


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


Total time for mapping row data: 109.67979598045349


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


...

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


Total time for mapping row data: 130.3154809474945
mu = 4.378351933353429


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


total time = 3499.049628019333


,Classifier,Train Error,Test Error,Std Error
1,"Linear SVM, C=1000.0",0.4531,0.4630,0.0634
2,"Gaussian SVM, C=1000.0, gamma =1000.0",0.0171,0.2358,0.0256
3,Decision Tree,0.0000,0.2336,0.0268
4,"RandomForestClassifier, n=100",0.0001,0.1598,0.0236
5,KNN,0.1637,0.2303,0.0300
6,Logistic Regression,0.6282,0.6361,0.0081
7,"AdaBoost, n=100",0.3327,0.3451,0.0227


In [42]:
LR = classification_init_Q(data, C=1, gamma=1, Q_size=20, mu_coeff=1, model='LR', 
                          epoch=50, init_iter=3, classifiers=clf, Leng=True, spd=True, 
                          accn=True, jrk=True, n_estimators=100, n_neighbors=5)
LR[0]

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specif

Total time for mapping row data: 108.39177393913269


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/lin

Total time for mapping row data: 110.94265675544739


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/lin

...

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/lin

Total time for mapping row data: 111.65236234664917
mu = 3.7814821232057625


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


total time = 3603.106202840805


,Classifier,Train Error,Test Error,Std Error
1,"Linear SVM, C=1000.0",0.4168,0.4179,0.0166
2,"Gaussian SVM, C=1000.0, gamma =1000.0",0.0094,0.2449,0.0138
3,Decision Tree,0.0000,0.2272,0.0146
4,"RandomForestClassifier, n=100",0.0007,0.1546,0.0101
5,KNN,0.1576,0.2145,0.0105
6,Logistic Regression,0.6337,0.6261,0.0109
7,"AdaBoost, n=100",0.3316,0.3449,0.0213
